In [8]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from proj1_helpers import *
from helpers import *
from implementations import *
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Load the training data into feature matrix, class labels, and event ids:

In [16]:
DATA_TRAIN_PATH = 'data/train.csv' 
y, x, ids = load_csv_data(DATA_TRAIN_PATH)

a = np.array([[1,2,3,8,5], [12, 23, 12, 9, 43], [0, 23, 8, 8, 4]])
a_3 = a[:, 3]
a_without_3 = a[:, np.array(range(a.shape[1])) != 3]

a_sep = []
for i in np.unique(a[:, 3]):
    a_sep.append(a_without_3[a_3 == i, :])
a_sep

In [42]:
sanitized_x = sanitize(x)
y = y.reshape((y.shape[0], 1))

feature_22 = sanitized_x[:, 22]
x_minus_22 = sanitized_x[:, np.array(range(sanitized_x.shape[1])) != 22]
x_sep = []
y_sep = []

for i in np.unique(feature_22):
    print(i)
    x_sep.append(x_minus_22[feature_22 == i, :])
    y_sep.append(y[feature_22 == i, :])


0.0
1.0
2.0
3.0


In [ ]:
accuracies = []
weights = []
for i, x_chunk in enumerate(x_sep):
    w_init = np.random.rand(x_chunk.shape[1], 1)
    w, loss = least_squares_GD(y_sep[i], x_chunk, w_init, 5000, 0.1, verbose=True) 
    y_pred = predict_labels(w, x_chunk)
    accuracies.append(compute_accuracy_linear_reg(y_sep[i], y_pred))
    weights.append(w)
    
accuracies

Iteration : 0 with loss 7.904257110444684 and gradient norm: 5.9145839170478895
Iteration : 250 with loss 0.2729126189015163 and gradient norm: 0.006538765436612333
Iteration : 500 with loss 0.2723851038093433 and gradient norm: 0.0032261998004240763
Iteration : 750 with loss 0.27222545231901113 and gradient norm: 0.0018971405518997612
Iteration : 1000 with loss 0.2721693309557731 and gradient norm: 0.001132208637022638
Iteration : 1250 with loss 0.27214927823770474 and gradient norm: 0.0006777310436834496
Iteration : 1500 with loss 0.27214208772700405 and gradient norm: 0.00040596948359918403
Iteration : 1750 with loss 0.27213950720027447 and gradient norm: 0.00024322117567043692
Iteration : 2000 with loss 0.2721385809199796 and gradient norm: 0.00014572241540480177
Iteration : 2250 with loss 0.27213824841615514 and gradient norm: 8.730826249046025e-05
Iteration : 2500 with loss 0.27213812905703216 and gradient norm: 5.2310066820892636e-05
Iteration : 2750 with loss 0.2721380862104963

In [43]:
prediction = x_sep[0] @ w
print(prediction)

prediction[prediction >= 0] = 1
prediction[prediction <= 0] = -1

print(prediction)

# JULES FLO: 81% accuracy sur le premier putain de merde: LE PLATEAU D'ARGENT
print(np.sum(prediction == y_sep[0])/y_sep[0].shape[0])

[[-0.62717446]
 [-0.351299  ]
 [-0.38718724]
 ...
 [-0.66857011]
 [ 0.09379569]
 [-0.53285079]]
[[-1.]
 [-1.]
 [-1.]
 ...
 [-1.]
 [ 1.]
 [-1.]]
0.8168006165363867


In [40]:
def accuracy_linear(y, x, w):
    return np.sum(x @ w == y)/y.shape[0]

In [41]:
for i in range(len(x_sep)):
    tx = x_sep[i]
    ty = y_sep[i]
    w, loss = least_squares_GD(ty, tx, w_init, 1000, 0.01, verbose=True)
    print(i,":")
    print("Accuracy on training set", accuracy_linear(ty, tx, w)) # JULES FLO: accuracy 0, bad dans mon calcul, pas le temps de fix

Iteration : 0 with loss 4.937772384969537 and gradient norm: 4.228983316573297
Iteration : 250 with loss 0.30602597127278447 and gradient norm: 0.18450464131426314
Iteration : 500 with loss 0.2784152240948924 and gradient norm: 0.05108569068152488
Iteration : 750 with loss 0.2751397856513302 and gradient norm: 0.025284386070332822
0 :
Accuracy on training set 0.0
Iteration : 0 with loss 6.083274826142171 and gradient norm: 5.10742363445162
Iteration : 250 with loss 0.42072091376084875 and gradient norm: 0.17261766310792043
Iteration : 500 with loss 0.3956973703869634 and gradient norm: 0.05214486448993269
Iteration : 750 with loss 0.39161957810698084 and gradient norm: 0.032763200935022975
1 :
Accuracy on training set 0.0
Iteration : 0 with loss 9.711191934803637 and gradient norm: 8.530077749262606
Iteration : 250 with loss 0.44595043035616466 and gradient norm: 0.2730022938015295
Iteration : 500 with loss 0.3765558422552624 and gradient norm: 0.09358339136680158
Iteration : 750 with 

In [33]:
for i in range(len(x_sep)):
    tx = x_sep[i]
    ty = y_sep[i]
    w, loss = logistic_regression(ty, tx, w_init, 1000, 0.1, verbose=True)
    print(i,":")
    print("Accuracy on training set", compute_accuracy(ty, tx, w))

Iteration : 0 with loss -1440136266.2580655 and gradient norm: 1.4853190575334942
Iteration : 250 with loss -252612012443.8168 and gradient norm: 0.9956756381694294
Iteration : 500 with loss -503765650455.6133 and gradient norm: 0.9956756337697791
Iteration : 750 with loss -754919535292.5062 and gradient norm: 0.9956756319783632
0 :
Accuracy on training set 0.7541060722828861
Iteration : 0 with loss -144833916.71591997 and gradient norm: 0.6946822409016756
Iteration : 250 with loss -9868258136.797886 and gradient norm: 0.2076441276726648
Iteration : 500 with loss -19625684870.420197 and gradient norm: 0.20764412015006836
Iteration : 750 with loss -29383313773.395008 and gradient norm: 0.2076441197989506
1 :
Accuracy on training set 0.6933482925822758
Iteration : 0 with loss -55361760.323068276 and gradient norm: 1.2373732212015471
Iteration : 250 with loss -7565402387.639501 and gradient norm: 0.2645289558469356
Iteration : 500 with loss -15018527388.389538 and gradient norm: 0.2645286

In [ ]:
w_init = np.random.rand(sanitized_x.shape[1], 1)
print(w_init.shape)
w, loss = least_squares_GD(sanitized_y, sanitized_x, np.random.rand(sanitized_x.shape[1], 1), 1000, 0.01, verbose=True)
print(w, loss)

In [ ]:
prediction = sanitized_x @ w
print(prediction)

prediction[prediction >= 0] = 1
prediction[prediction <= 0] = -1

print(prediction)

print(np.sum(prediction == sanitized_y)/sanitized_y.shape[0])

In [ ]:
print(compute_accuracy(sanitized_y, sanitized_x, ))

Reduce dimensions and standardize the columns of X

Running logistic regression with different values of degrees (polynomial feature expansion), gamma and different initial weights

In [ ]:
print(y_train.shape, tx_train.shape, w.shape)

In [ ]:
N = sanitized_x.shape[0]
inds = np.random.choice(range(N), 100000, replace=False)
sanitized_x_sub = sanitized_x[inds, :]
sanitized_x_sub = sanitized_x_sub[:, [0,1,2,3,7,8,9,10,11,13,14,17,19,21,22,29]]
sanitized_y_sub = sanitized_y[inds]
x_train, x_test, y_train, y_test = split_data(sanitized_x, sanitized_y, 0.8)
number_of_w = 1

for degree in range(1, 5)
    tx_train = build_poly(x_train, degree)
    tx_test = build_poly(x_test, degree)
    for gamma in np.array([0.01, 0.05, 0.1, 0.5]):
        print("Running logistic regression with polynomial feature expansion with degree", degree, 
                 "and gamma", gamma,", with", number_of_w, "different initial weights.")
        for i in  range(number_of_w):
            w_init = np.random.rand(tx_train.shape[1], 1)
            w, loss = logistic_regression(y_train, tx_train, w_init, 1000, gamma, verbose=True)
            print(i,":")
            print("Accuracy on training set", compute_accuracy(y_train, tx_train, w))
            print("Accuracy on test set:, ", compute_accuracy(y_test, tx_test, w))

In [ ]:
np.unique(x[:, 22])

In [ ]:
N = sanitized_x.shape[0]
inds = np.random.choice(range(N), 100000, replace=False)
sanitized_x_sub = sanitized_x[inds, :]
sanitized_x_sub = sanitized_x_sub[:, [0,1,2,3,7,8,9,10,11,13,14,17,19,21,22,29]]
sanitized_y_sub = sanitized_y[inds]
x_train, x_test, y_train, y_test = split_data(sanitized_x, sanitized_y, 0.8)
number_of_w = 1

sanitized_x[:, 22]

for degree in range(1, 5)
    tx_train = build_poly(x_train, degree)
    tx_test = build_poly(x_test, degree)
    for gamma in np.array([0.01, 0.05, 0.1, 0.5]):
        print("Running logistic regression with polynomial feature expansion with degree", degree, 
                 "and gamma", gamma,", with", number_of_w, "different initial weights.")
        for i in  range(number_of_w):
            w_init = np.random.rand(tx_train.shape[1], 1)
            w, loss = logistic_regression(y_train, tx_train, w_init, 1000, gamma, verbose=True)
            print(i,":")
            print("Accuracy on training set", compute_accuracy(y_train, tx_train, w))
            print("Accuracy on test set:, ", compute_accuracy(y_test, tx_test, w))

## Generate predictions and save ouput in csv format for submission:


In [ ]:
DATA_TEST_PATH = '' # TODO: download train data and supply path here 
_, tX_test, ids_test = load_csv_data(DATA_TEST_PATH)

In [ ]:
OUTPUT_PATH = '' # TODO: fill in desired name of output file for submission
y_pred = predict_labels(weights, tX_test)
create_csv_submission(ids_test, y_pred, OUTPUT_PATH)

In [ ]:
a = np.random.rand(30)
np.around(a)

In [ ]:
a